### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

# A notebook to test running RBAs at the voxel level on fMRI data.
Brainhack-Aus 2022 project

Authors:
Gang Chen @afni-gangc
Christopher Nolan @crnolan
Kelly Garner @kel-github 
Lea Waller @HippocampusGirl
Daniel Tomasz @danieltomasz
Megan Campbell @meganEJcampbell
Preetom Pal @preqon
Adam @a-manoogian *
Bella @isabellaorlando *
Darin Leiter @dsleiter *
Arshiyan @Arshiyasan *
Judy Zhu @jd-zhu 


Modelling task-based fMRI data often involves performing a GLM at each voxel and then correcting for many many many multiple comparisons.

Here instead, we try performing a single hierarchical mixed effects model on all the voxels at once.

This provides advantages typical of Bayesian hiearchal modelling; information at upper levels of the hierarchy (e.g. across voxels) can help inform estimates at lower levels (the estimate for each voxel) - aka shrinkage - and we avoid the multiple comparisons problem by instead providing the strength of evidenve for the effect of interest at each voxel.

For a comprehensive introduction to this approach, see this paper and this paper by Gang Chen.

Here we test the feasibility of running Bayesian hierarchal modelling at the voxel level, by determining compute time across varying data sizes; both randomly generated and fMRI data.

# Running the notebook

To build the environment to run this notebook, follow the instructions [here](https://github.com/crnolan/pyrba)

# Import modules

In [1]:
import arviz as az
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import numpy as np
import pandas as pd
import pymc as pm
import xarray as xr
import bambi as bmb
sns.set_theme(style='white')

# Generate data

Here is an example of how to generate a test dataset. We are simulating voxel data from a task with 2 conditions (e.g. go and stop conditions of a stop signal task), with additive values per voxel and per subject. 

In [2]:
# define the number of voxels and subjects for which you wish to simulate data
nvoxels = 1000
nsubs = 30
mean_cond_a = .5
mean_cond_b = .3
sd_cond_a = .15
sd_cond_b = .15

In [3]:
#create voxel noise parameters
noise_list = []
for i in range(0,nvoxels): #hardcoded
    mean =  np.random.normal(0, .08) #voxel noise paramerer - normal distribution
    sd =  abs(np.random.normal(0, .05)) #voxel noise paramerer - uniform distribution or absolute normal distribution
    noise_list.append([mean, sd])


In [4]:
noise_list

[[-0.07953100439618851, 0.04545793670293904],
 [-0.00724696417374766, 0.007833597661327556],
 [0.0359430574324949, 0.0287049113669399],
 [-0.05547943199512097, 0.068634687734844],
 [0.02985574036490054, 0.02871853492902675],
 [0.04920993146015775, 0.09201911025111985],
 [0.06550086473800425, 0.023149538246841288],
 [-0.03632252820133593, 0.018024362096888784],
 [-0.043934737384787786, 0.048308785836757615],
 [0.005405758926370034, 0.06090926437245822],
 [0.050556212404230395, 0.020342639744276938],
 [0.07032466007448636, 0.016021492545644177],
 [-0.16780343193155392, 0.05437774114603064],
 [0.10620306487153773, 0.13939220489913562],
 [-0.05025138008708896, 0.009835124877944988],
 [0.04787705354084043, 0.02406457613713628],
 [-0.15506923547597237, 0.034333399504129856],
 [0.08137417460284656, 0.03611597683825958],
 [-0.008554096728544291, 0.07482178373054187],
 [0.13000277256930812, 0.02422292044933326],
 [0.07977945985087896, 0.007179363479408453],
 [-0.03441417095397666, 0.01830636009

In [5]:
#define function to generate random voxel values
def generate_random_voxels(mean, sd, noise_list, length=nvoxels): #hardcoded
    # mean [1 value] - reflects the mean of the condition + subject
    # sd [as above, but sd]
    # noise_list arr[nvoxels, 2] rfx for each voxel
    voxels = []
    for v in range(length):
        mean = mean + noise_list[v][0] + np.random.normal(0, 0.2)# mean = condition + subrfx + voxel mean + residual noise
        sd = sd + noise_list[v][1] # as above but with sd
        voxels.append(np.random.normal(mean,sd)) 
    return voxels

In [6]:
participants = []
conditions = []

In [7]:
#create multi level index matrix
for i in range(nsubs):
    participants.append(i)
    participants.append(i)
    conditions.append(0)
    conditions.append(1)
arrays = [participants, conditions]
tuples = list(zip(*arrays))
multi_index = pd.MultiIndex.from_tuples(tuples, names=["participant", "condition"])

In [8]:
#initiate voxel list
data = np.zeros((nsubs*2,nvoxels)) #hardcoded to be *2 participants (for 2 conditions)
df = pd.DataFrame(data, index = multi_index)

In [9]:
data.shape

(60, 1000)

In [10]:
#populate the multi level index matrix
for participant in range(nsubs):
    # unique number for each paritipcant
    random_effect_mean = np.random.normal(0, .1) # drawn from common distribution
    random_effect_sd = abs(np.random.normal(0, .05))
    
    for condition in range(2): # 2 conditions
        if condition == 0:
            mean = mean_cond_a + random_effect_mean
            sd = sd_cond_a + random_effect_sd
            df.loc[participant, condition] = generate_random_voxels(mean,sd,noise_list) 
        if condition == 1:
            mean = mean_cond_b + random_effect_mean
            sd = sd_cond_b + random_effect_sd
            df.loc[participant, condition] = generate_random_voxels(mean,sd,noise_list) 

In [11]:
df

0         1         2         3         4    \
participant condition                                                     
0           0          0.930762  0.978346  1.964742  0.915747  1.653046   
            1          0.202021  0.790483  0.576306  0.824341  0.877298   
1           0          0.670959  0.512161  0.079051  0.853072  0.448462   
            1          0.038567  0.106768  0.414091 -0.064505  0.313942   
2           0          0.785029  0.518225  0.498828  0.452685  0.386068   
            1          0.689356  0.941617  1.895802  1.758050  0.951013   
3           0          1.075865  1.618171  1.823876  0.970041  0.826132   
            1          0.142324 -0.193734 -0.168920 -0.038075 -0.240468   
4           0          0.598515  0.738268  0.633810  0.239825  0.949515   
            1          0.091647  0.451676  0.514468  0.626707  0.265814   
5           0          0.206925  0.453180  0.598091  0.895473  0.870889   
            1         -0.045971  0.464119  0.432538  0.268782  0.445739   
6           0          0.019514  0.091109  0.550304 -0.025568 -0.073258   
            1          0.445722  0.347475  1.048701  0.330852 -0.112639   
7           0          0.199142  0.629540  0.069248  0.478795  0.205121   
            1          0.819598  0.839548  0.738412  0.731249  0.785257   
8           0          0.530634  0.808659  0.466913  0.542559  0.570025   
            1          0.302524  0.255160  0.419795  0.432946  0.707834   
9           0          0.231827  0.638366  0.225051  0.017927  0.107441   
            1         -0.631455 -0.177572 -0.465684 -0.397082 -1.033940   
10          0          0.428147  0.611664  0.868219  0.146105  1.035665   
            1          0.646090  0.503567  1.197877  0.306445  0.074446   
11          0          1.087423  0.655354  0.467330  0.900763  1.145982   
            1          0.163234 -0.251858 -0.093363 -0.583889 -0.708273   
12          0          0.466062 -0.258108 -0.442389  0.394859  0.397744   
            1          0.258113 -0.359588  0.083376 -0.574190  0.073250   
13          0          0.092058  0.326289 -0.226782 -0.346546 -0.736702   
            1         -0.033330  0.262136  0.549986  0.683590  0.316017   
14          0         -0.003991  0.098187 -0.227552  0.002329  0.020945   
            1         -0.031451  0.315492 -0.881374 -0.335842 -0.447138   
15          0          0.720558  0.979058  0.597777  1.309287  1.313278   
            1          0.285894  0.369176  0.973350  0.225432  0.064192   
16          0         -0.122728 -0.058716  0.656475  1.108536  0.138413   
            1          0.373484  0.350001 -0.255589 -0.083292 -0.401079   
17          0          0.715320  0.945864  0.758003  0.544221  0.537508   
            1          0.322777  0.343325  0.916044  0.673975  0.074226   
18          0          0.672245  0.931732  1.330221  1.107750  1.848755   
            1          1.036883  1.009076  0.745881  0.753964 -0.188061   
19          0          0.684763  0.099653  0.633193  1.403919  0.874734   
            1         -0.138262 -0.535593 -0.593484 -1.342106 -0.586239   
20          0          0.193314  0.051433  0.756375  0.522421  0.215812   
            1         -0.164415  0.457570  0.794049 -0.042883 -0.193501   
21          0          0.654286  0.126932  0.808973  1.337416  1.266901   
            1          0.412051  0.504881  0.195317  0.140865  0.578483   
22          0          0.662232 -0.278435  0.313192  1.113097 -0.073653   
            1         -0.026614  0.044871 -0.359569 -0.421938 -0.840031   
23          0          0.631981  0.685497  0.781170  0.127692  1.460256   
            1         -0.275208  0.012657  0.187316  0.222517  0.541070   
24          0          0.422978  0.494223  0.725436  0.635103  0.349589   
            1         -0.001201  0.774991  0.254429 -0.234348  0.056150   
25          0          0.380168  0.761731  0.205409  0.726064  1.086380   
            1          0.167766  0.458249  0.486053  

In [12]:
#melt to satisfy bambi long form
df = pd.melt(df, ignore_index=False, var_name="voxel_id", value_name = "BOLD")

df


voxel_id       BOLD
participant condition                     
0           0                 0   0.930762
            1                 0   0.202021
1           0                 0   0.670959
            1                 0   0.038567
2           0                 0   0.785029
...                         ...        ...
27          1               999  -9.583776
28          0               999  -2.823201
            1               999  14.581491
29          0               999  35.614322
            1               999  24.391166

[60000 rows x 2 columns]

In [13]:
#write to csv
tmp = df


In [14]:
tmp.head(5)

voxel_id      BOLD
participant condition                    
0           0                 0  0.930762
            1                 0  0.202021
1           0                 0  0.670959
            1                 0  0.038567
2           0                 0  0.785029

In [15]:
tmp.to_csv('df2.txt', sep=' ')

In [16]:
df = pd.read_csv('df2.txt', delimiter = ' ')
df.head(5)

,participant,condition,voxel_id,BOLD
0,0,0,0,0.930762
1,0,1,0,0.202021
2,1,0,0,0.670959
3,1,1,0,0.038567
4,2,0,0,0.785029


Now I want to define the following model to apply to the data:


In [17]:
model = bmb.Model("BOLD ~ condition + (1|participant) + (1|voxel_id)", data=df)

In [ ]:
%%time

fitted = model.fit(tune=4000, 
                   draws=1000, 
                   chains=16, 
                   method='nuts_numpyro',
                   nuts_kwargs=dict(max_tree_depth=100))

/Users/daniel/.pyenv/versions/voxel-bayes-3.12/lib/python3.12/site-packages/bambi/models.py:323: FutureWarning: the method argument has been deprecated, please use inference_method
  warnings.warn(
/Users/daniel/.pyenv/versions/voxel-bayes-3.12/lib/python3.12/site-packages/bambi/backend/pymc.py:116: FutureWarning: 'nuts_numpyro' has been replaced by 'numpyro_nuts' and will be removed in a future release
  warnings.warn(
warmup:   2%|▏         | 33/1500 [02:37<2:20:21,  5.74s/it]

In [ ]:
fitted = model.fit(tune=4000, 
                   draws=1000, 
                   chains=16, 
                   method='nutpie',
                   nuts_kwargs=dict(max_tree_depth=100))

In [ ]:
model.graph()

In [ ]:
az.plot_trace(fitted, figsize=(20, 35))
az.summary(fitted)